In [44]:
%load_ext autoreload
%autoreload 2
from utils import *
from datasets import load_dataset
from transformers import (
    HfArgumentParser, 
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer
)
from link import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
with open('boolq_perturbed.json', 'r') as f:
  json_data = json.load(f)
bundles = convert_bundles_to_yn(json_data)
with open('pubmed_test.json', 'r') as f:
  pubmed_test = json.load(f)
pubmed = load_dataset('qiaojin/PubMedQA', 'pqa_labeled')
pubmedds = convert_pubmed_to_yn(pubmed, list(map(int, list(pubmed_test.keys()))))


Label distribution in train split:
yes: 276 (62.02%)
no: 169 (37.98%)
Total examples in train: 445

Label distribution in test split:
yes: 276 (62.02%)
no: 169 (37.98%)
Total examples in test: 445


In [48]:
# dataset = load_dataset('bigbio/bioasq_task_b')
#ds = load_dataset("kroshan/BioASQ")
with open('datasets/BioASQ-training12b/training12b_new.json', 'r') as f:
  bio_train_data = json.load(f)
with open('datasets/Task12BGoldenEnriched/12B1_golden.json', 'r') as f:
  bio_test_data1 = json.load(f)
with open('datasets/Task12BGoldenEnriched/12B2_golden.json', 'r') as f:
  bio_test_data2 = json.load(f)
with open('datasets/Task12BGoldenEnriched/12B3_golden.json', 'r') as f:
  bio_test_data3 = json.load(f)
with open('datasets/Task12BGoldenEnriched/12B4_golden.json', 'r') as f:
  bio_test_data4 = json.load(f)
golden = [bio_test_data1, bio_test_data2, bio_test_data3, bio_test_data4]
biods = convert_bioasq_to_yn(bio_train_data, golden)


In [97]:

with open('boolq_perturbed.json', 'r') as f:
  json_data = json.load(f)
contrast = convert_contrast_to_yn(json_data)

In [102]:
total_lens = 0
ab256 = 0
ab512 = 0
for ex in pubmedds['train']:
  lenex = len(ex['context'].split(' '))
  total_lens += lenex
  if(lenex>512):
    ab512+=1
  if(lenex>256):
    ab256+=1
print(total_lens/445)
print(ab256)
print(ab512)

196.79775280898878
44
0


In [73]:
new = {split: {key: pubmedds[split][0][key] for key in ['question', 'context', 'label', 'id']} for split in pubmedds.keys()}
new

{'train': {'question': 'Can tailored interventions increase mammography use among HMO women?',
  'context': 'Telephone counseling and tailored print communications have emerged as promising methods for promoting mammography screening. However, there has been little research testing, within the same randomized field trial, of the efficacy of these two methods compared to a high-quality usual care system for enhancing screening. This study addressed the question: Compared to usual care, is tailored telephone counseling more effective than tailored print materials for promoting mammography screening? Three-year randomized field trial. One thousand ninety-nine women aged 50 and older recruited from a health maintenance organization in North Carolina. Women were randomized to 1 of 3 groups: (1) usual care, (2) tailored print communications, and (3) tailored telephone counseling. Adherence to mammography screening based on self-reports obtained during 1995, 1996, and 1997. Compared to usual 

In [35]:
tokenizer = AutoTokenizer.from_pretrained('./output/yn_models/boolq-art')

train_dataset = biotrain['train']
train_dataset = train_dataset.select(range(128))

train_dataset = train_dataset.map(
    lambda x: prepare_dataset_yn(x, tokenizer, 128),
    batched=True,
    remove_columns=train_dataset.column_names
)

Map: 100%|██████████| 128/128 [00:00<00:00, 347.82 examples/s]


In [90]:
ex = 'All but two of the universities in the Russell Group are part of the Sutton Trusts group of 30 highly selective universities, the Sutton Trust 30 (the absent members being Queen Mary University of London and Queens University Belfast). The Sutton 13 group of the 13 most highly selective universities only includes one non-Russell Group member, the University of St Andrews. St Andrews was also the only non-Russell Group University in the top 10 by average UCAS tariff score of new undergraduate students in 2015--16, placing fifth with an average score of 525 (and an offer rate of 52.2%). Half of the Russell Group made offers to more than three quarter of their undergraduate applicants in 2015.'

In [91]:
len(ex.split(' '))

117

In [22]:
control = {
  "total_f1": 0.5712298558100084,
  "total_exact_match": 0.601123595505618,
  "total_precision": 0.576271186440678,
  "total_recall": 0.5710806897459524,
  "yes_f1": 0.6844444444444444,
  "yes_precision": 0.652542372881356,
  "yes_recall": 0.719626168224299,
  "yes_exact_match": 0.43258426966292135,
  "no_f1": 0.4580152671755725,
  "no_precision": 0.5,
  "no_recall": 0.4225352112676056,
  "no_exact_match": 0.16853932584269662
}

exp = {
  "total_f1": 0.617049885706602,
  "total_exact_match": 0.6404494382022472,
  "total_precision": 0.621256038647343,
  "total_recall": 0.6156377517441095,
  "yes_f1": 0.7117117117117117,
  "yes_precision": 0.6869565217391305,
  "yes_recall": 0.7383177570093458,
  "yes_exact_match": 0.4438202247191011,
  "no_f1": 0.5223880597014925,
  "no_precision": 0.5555555555555556,
  "no_recall": 0.49295774647887325,
  "no_exact_match": 0.19662921348314608
}

In [25]:
res = {key: exp[key] - control[key] for key in control.keys()}

In [26]:
res

{'total_f1': 0.04582002989659362,
 'total_exact_match': 0.0393258426966292,
 'total_precision': 0.04498485220666504,
 'total_recall': 0.04455706199815712,
 'yes_f1': 0.027267267267267248,
 'yes_precision': 0.034414148857774496,
 'yes_recall': 0.01869158878504673,
 'yes_exact_match': 0.011235955056179747,
 'no_f1': 0.06437279252591999,
 'no_precision': 0.05555555555555558,
 'no_recall': 0.07042253521126762,
 'no_exact_match': 0.02808988764044945}